In [ ]:
# !pip install sumy
!pip install requests
!pip install beautifulsoup4
!pip install spacy
!pip install tqdm
!pip install ipynbname
!pip install -q -U google-generativeai
!pip install pandas

In [ ]:
import requests
from bs4 import BeautifulSoup
import urllib
import json
import spacy
from collections import deque
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest

In [ ]:
import google.generativeai as genai
import random

api_keys = [
    "API_KAY_1",
]


def machinetranslator(input, prompt, wordlength):
    for _ in range(5):
        try:
            genai.configure(api_key=random.choice(api_keys))
            model = genai.GenerativeModel('gemini-pro')
            messages = [
                {'role': 'user',
                'parts': [f"{prompt}, within {wordlength} words:{input}"]}
            ]
            response = model.generate_content(messages)
            return response.text
        except Exception as e:
            pass
    return "Translation failed after 5 attempts."
def gemini_summary(text, wordlength=300):
  prompt = "Provide a concise summary of the given text, highlighting its main arguments, key points, and central themes, in one paragraph, without any newline"
  return machinetranslator(text, prompt, wordlength)
# print(gemini_summary(text,300))

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
def fetch_and_parse(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup.get_text()

In [ ]:
api_key = "API_KEY"
cse_id = "your_cse_id"

# The sentence want to search
query = "Albert Einstein"

def google_search(query, api_key, cse_id, **kwargs):
    service_url = 'https://www.googleapis.com/customsearch/v1'
    params = {
        'q': query,
        'key': api_key,
        'cx': cse_id,
    }
    params.update(kwargs)
    response = requests.get(service_url, params=params)
    return response.json()

In [ ]:
def dfs_search(query, api_key, cse_id, depth=0, search_res=[]):
    depth += 1
    if depth>2:
        return search_res
    results = google_search(query, api_key, cse_id)
    search_res.extend(results.get('items', []))
    for result in results.get('items', [])[:3]:
        return dfs_search(result['title'], api_key, cse_id, depth, search_res)

In [ ]:
# DFS search function
search_results = dfs_search(query, api_key, cse_id, 0)

# Print DFS the search results
for result in search_results:
     print(result['title'], result['link'])

Albert Einstein - Wikipedia https://en.wikipedia.org/wiki/Albert_Einstein
Albert Einstein – Biographical - NobelPrize.org https://www.nobelprize.org/prizes/physics/1921/einstein/biographical/
Albert Einstein | Biography, Education, Discoveries, & Facts ... https://www.britannica.com/biography/Albert-Einstein
Albert Einstein College of Medicine | Medical Education | Biomedical ... https://einsteinmed.edu/
Albert Einstein https://www.history.com/topics/inventions/albert-einstein
Albert Einstein: His life, theories and impact on science | Space https://www.space.com/15524-albert-einstein.html
Albert Einstein HS https://www.montgomeryschoolsmd.org/schools/einsteinhs/
Einstein | Jefferson Health https://www.jeffersonhealth.org/einstein
The Official Licensing Site of Albert Einstein http://einstein.biz/
Albert Einstein: Biography, Physicist, Nobel Prize Winner https://www.biography.com/scientists/albert-einstein
Albert Einstein - Wikipedia https://en.wikipedia.org/wiki/Albert_Einstein
Albert

In [ ]:
def summary(search_result):
    text = ""
    for result in search_result:
        text += fetch_and_parse(result['link'])
    text = text.replace('   ', '').replace('\n', '')
    doc = nlp(text[-100000:])
    print(f'Number of sentences sent: {len(list(doc.sents))}')
    keyword = []
    stopwords = list(STOP_WORDS)
    pos_tag = ['PROPN', 'ADJ', 'NOUN', 'VERB']
    for token in doc:
        if(token.text in stopwords or token.text in punctuation):
            continue
        if(token.pos_ in pos_tag):
            keyword.append(token.text)


    freq_word = Counter(keyword)
    print("Most Frequent Words:")
    print(freq_word.most_common(5))


    max_freq = Counter(keyword).most_common(1)[0][1]
    for word in freq_word.keys():
            freq_word[word] = (freq_word[word]/max_freq)
    print(freq_word.most_common(5))



    sent_strength={}
    for sent in doc.sents:
        for word in sent:
            if word.text in freq_word.keys():
                if sent in sent_strength.keys():
                    sent_strength[sent]+=freq_word[word.text]
                else:
                    sent_strength[sent]=freq_word[word.text]

    #nlargest function returns a list containing the top 3 sentences
    summarized_sentences = nlargest(3, sent_strength, key=sent_strength.get)

    final_sentences = [ w.text for w in summarized_sentences ]
    summary = ' '.join(final_sentences)
    return summary, text

In [ ]:
print('DFS SUMMARY:')
summ, text_dfs = summary(search_results)
print(f'Summary:\n{summ}')

DFS SUMMARY:
Number of sentences sent: 739
Most Frequent Words:
[('Einstein', 264), ('p.', 65), ('theory', 47), ('time', 42), ('quote', 42)]
[('Einstein', 1.0), ('p.', 0.24621212121212122), ('theory', 0.17803030303030304), ('time', 0.1590909090909091), ('quote', 0.1590909090909091)]
Summary:
Eliezer Yudkowsky, "Einstein's Speed" (21 May 2008)See also[edit]Albert Einstein and politicsAnnus Mirabilis papersEPR paradoxOn the Method of Theoretical Physics, Einstein's Herbert Spencer lecture at Oxford (June 10, 1933).The Meaning of Relativity (1922 book consisting of an English translation of four lectures given by Einstein in German at Princeton University in May 1921; with several subsequent editions)Theory of relativityUnified field theoryWhy Socialism?Bohr–Einstein debatesPhilosophy of scienceConceptsAnalysis • A priori and a posteriori • Causality • Demarcation problem • Fact • Inductive reasoning • The earliest published attribution of this quote to Einstein found on Google Books is t

In [ ]:
print('GEMINI SUMMARY of DFS:')
short = gemini_summary(text_dfs[-10000:])
print(f'Summary:\n{short}')

GEMINI SUMMARY of DFS:
Summary:
The text delves into the scientific and philosophical perspectives of Albert Einstein, highlighting his unique approach to understanding the universe. It emphasizes his relentless pursuit of fundamental truths, leading him to question established theories and seek out unsettling situations. While his colleagues made progress by exploiting existing ideas, Einstein aimed to transcend them, resulting in his aloofness from the applications of quantum theory. The author discusses Einstein's attempt to introduce a cosmological constant to create a static universe model, which later proved obsolete with the discovery of the universe's expansion. It also mentions Einstein's understanding of general relativity's central concepts before developing the detailed equations. Additionally, the text explores Einstein's perspective on capitalism, arguing that the owner of the means of production determines the worker's payment based on minimum needs and labor requirement

In [ ]:
def bfs_search(query, api_key, cse_id):
    queue = deque([query])
    visited = set()
    search_results = []
    node_cnt = 0
    while queue:
        node_cnt += 1
        if(node_cnt>5):
            break
        current_query = queue.popleft()
        if current_query not in visited:
            visited.add(current_query)
            results = google_search(current_query, api_key, cse_id)
            search_results.extend(results.get('items', []))
            # Add related queries to the queue
            for result in results.get('items', []):
                queue.append(result['title'])
    return search_results


In [ ]:
# BFS search function
search_results = bfs_search(query, api_key, cse_id)

# Print the search results
for result in search_results:
    print(result['title'], result['link'])

Albert Einstein - Wikipedia https://en.wikipedia.org/wiki/Albert_Einstein
Albert Einstein – Biographical - NobelPrize.org https://www.nobelprize.org/prizes/physics/1921/einstein/biographical/
Albert Einstein | Biography, Education, Discoveries, & Facts ... https://www.britannica.com/biography/Albert-Einstein
Albert Einstein College of Medicine | Medical Education | Biomedical ... https://einsteinmed.edu/
Albert Einstein https://www.history.com/topics/inventions/albert-einstein
Albert Einstein: His life, theories and impact on science | Space https://www.space.com/15524-albert-einstein.html
Albert Einstein HS https://www.montgomeryschoolsmd.org/schools/einsteinhs/
Einstein | Jefferson Health https://www.jeffersonhealth.org/einstein
The Official Licensing Site of Albert Einstein http://einstein.biz/
Albert Einstein: Biography, Physicist, Nobel Prize Winner https://www.biography.com/scientists/albert-einstein
Albert Einstein - Wikipedia https://en.wikipedia.org/wiki/Albert_Einstein
Albert

In [ ]:
print('BFS SUMMARY:')
summ, text_bfs = summary(search_results[20: 40])
print(f'Summary:\n{summ}')

BFS SUMMARY:
Number of sentences sent: 802
Most Frequent Words:
[('Einstein', 323), ('Albert', 75), ('theory', 54), ('identifiersArticles', 47), ('relativity', 34)]
[('Einstein', 1.0), ('Albert', 0.23219814241486067), ('theory', 0.16718266253869968), ('identifiersArticles', 0.14551083591331268), ('relativity', 0.10526315789473684)]
Summary:
investigationsWave–particle dualityGravitational waveTea leaf paradoxWorksAnnus mirabilis papers (1905)"Investigations on the Theory of Brownian Movement" (1905)Relativity: The Special and the General Theory (1916)The Meaning of Relativity (1922)The World as I See It (1934)The Evolution of Physics (1938)"Why Socialism?" (1949)Russell–Einstein Manifesto (1955)In popularcultureDie Grundlagen der Einsteinschen Relativitäts-Theorie (1922 documentary)The Einstein Theory of Relativity (1923 documentary)Relics: Einstein's Brain (1994 documentary)Insignificance (1985 film)Picasso at the Lapin Agile (1993 play)I.Q. (1994 film)Einstein's Gift (2003 play)Einst

In [ ]:
print('GEMINI SUMMARY of BFS:')
short = gemini_summary(text_bfs[-10000:])
print(f'Summary:\n{short}')

GEMINI SUMMARY of BFS:
Summary:
Einstein's College of Medicine, founded in 1953, is renowned for its commitment to medical education, biomedical research, and patient care. It has over 2,000 full-time faculty, 737 M.D. students, and 209 Ph.D. students. The college has a long history of innovation, being among the first to integrate bedside experience with learning, develop bioethics as an academic discipline, and establish a residency program in internal medicine with a focus on women's health. Its research programs have received significant funding from the National Institutes of Health, with a focus on aging, developmental brain research, cardiac disease, and ethnic and racial health disparities. The college's partnership with Montefiore Medical Center enhances clinical and translational research, leading to accelerated adoption of new discoveries for patient treatments. With its dedication to excellence in education, research, and patient care, Einstein's College of Medicine continu